In [77]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os
import time
import pandas as pd
import re

In [78]:
load_dotenv()

True

In [79]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


In [80]:
# function to crawl the website and get its content
def crawl_website(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Failed to fetch the website. Status code: {response.status_code}")


In [81]:
def parse_website(content):
    soup = BeautifulSoup(content, 'html.parser')
    data = {
        "title": soup.title.string if soup.title else None,
        "meta_description": soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else None,
        "links": [a['href'] for a in soup.find_all('a', href=True)]
    }
    return data

In [96]:
def extract_links(df):
    link_pattern = r'https?://[^\s,]+'
    
    # Ensure that we apply the regex only to strings
    df['extracted_links'] = df['links'].apply(lambda x: re.findall(link_pattern, str(x)) if isinstance(x, str) else [])
    
    return df

def split_links_into_rows(df):
    link_pattern = r'https?://[^\s,]+'
    df['extracted_links'] = df['links'].apply(lambda x: re.findall(link_pattern, x))
    df_exploded = df.explode('extracted_links', ignore_index=True)    
    return df_exploded[['title', 'meta_description', 'extracted_links']]


def store_data_in_db(data):
    try:
        conn = psycopg2.connect(
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        cursor.execute(''' 
            CREATE TABLE IF NOT EXISTS website_data (
                id SERIAL PRIMARY KEY,
                title TEXT,
                meta_description TEXT,
                links TEXT[] 
            );
        ''')

        title_list = data['title'].tolist()
        meta_description_list = data['meta_description'].tolist()
        links_list = data['extracted_links'].tolist()

        for title, meta_description, links in zip(title_list, meta_description_list, links_list):
            formatted_links = '{' + ','.join([f'"{link}"' for link in links]) + '}'

            cursor.execute(''' 
                INSERT INTO website_data (title, meta_description, links) 
                VALUES (%s, %s, %s)
            ''', (title, meta_description, formatted_links))

        conn.commit()
    except Exception as e:
        print(f"Error storing data in the database: {e}")
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    url = "https://akshayakalpa.org/"

    try:
        website_content = crawl_website(url)
        parsed_data = parse_website(website_content)
        links_df = pd.DataFrame({'links': [parsed_data['links']]})
        links_df['links'] = links_df['links'].apply(lambda x: ','.join(str(i) for i in x))
        links_df = extract_links(links_df)
        links_df['title'] = parsed_data['title']
        links_df['meta_description'] = parsed_data['meta_description']
        links_df_exploded = split_links_into_rows(links_df)
        store_data_in_db(links_df_exploded)

        print("Data has been successfully stored in the database.")
    except Exception as e:
        print(f"An error occurred: {e}")


Data has been successfully stored in the database.


In [ ]:


# Function to store data in PostgreSQL database

